In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import  DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision import transforms


In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),  # converts PIL Image -> Tensor, scales 0-255 to 0-1
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # normalize RGB channels
])

In [3]:
#Lets first download/get the daat
train_data= CIFAR10(
    root='data',
    download= True,
    train= True,
    transform= transform,
    target_transform= None
)

test_data= CIFAR10(
    root='data',
    download= True,
    transform= transform,
    train=False
)

100%|██████████| 170M/170M [00:06<00:00, 26.1MB/s] 


Lets see what the data looks like

In [4]:
train_data[0][0]

tensor([[[-0.5373, -0.6627, -0.6078,  ...,  0.2392,  0.1922,  0.1608],
         [-0.8745, -1.0000, -0.8588,  ..., -0.0353, -0.0667, -0.0431],
         [-0.8039, -0.8745, -0.6157,  ..., -0.0745, -0.0588, -0.1451],
         ...,
         [ 0.6314,  0.5765,  0.5529,  ...,  0.2549, -0.5608, -0.5843],
         [ 0.4118,  0.3569,  0.4588,  ...,  0.4431, -0.2392, -0.3490],
         [ 0.3882,  0.3176,  0.4039,  ...,  0.6941,  0.1843, -0.0353]],

        [[-0.5137, -0.6392, -0.6235,  ...,  0.0353, -0.0196, -0.0275],
         [-0.8431, -1.0000, -0.9373,  ..., -0.3098, -0.3490, -0.3176],
         [-0.8118, -0.9451, -0.7882,  ..., -0.3412, -0.3412, -0.4275],
         ...,
         [ 0.3333,  0.2000,  0.2627,  ...,  0.0431, -0.7569, -0.7333],
         [ 0.0902, -0.0353,  0.1294,  ...,  0.1608, -0.5137, -0.5843],
         [ 0.1294,  0.0118,  0.1137,  ...,  0.4431, -0.0745, -0.2784]],

        [[-0.5059, -0.6471, -0.6627,  ..., -0.1529, -0.2000, -0.1922],
         [-0.8431, -1.0000, -1.0000,  ..., -0

In [5]:
train_data[0][1]

6

In [6]:
train_data.classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [7]:
len(test_data)

10000

In [8]:
BATCH_SIZE= 64
train_data_batched= DataLoader(
    train_data,
    BATCH_SIZE,
    shuffle=True
)
test_data_batched= DataLoader(
    test_data,
    BATCH_SIZE,
    shuffle=False
)

In [9]:
class ImageClassifierCNN(nn.Module):
    def __init__(self, input_shape, output_shape, hidden_unit):
        super().__init__()
        
        self.block1 = nn.Sequential(
            nn.Conv2d(input_shape, hidden_unit, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm2d(hidden_unit),
            nn.ReLU(),
            nn.Conv2d(hidden_unit, hidden_unit, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm2d(hidden_unit),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.block2 = nn.Sequential(
            nn.Conv2d(hidden_unit, hidden_unit*2, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm2d(hidden_unit*2),
            nn.ReLU(),
            nn.Conv2d(hidden_unit*2, hidden_unit*2, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm2d(hidden_unit*2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.block3 = nn.Sequential(
            nn.Conv2d(hidden_unit*2, hidden_unit*4, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm2d(hidden_unit*4),
            nn.ReLU(),
            nn.Conv2d(hidden_unit*4, hidden_unit*4, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm2d(hidden_unit*4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classification_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_unit * 4 * 4 * 4, hidden_unit * 4),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_unit * 4, output_shape)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.classification_layer(x)
        return x

In [10]:
device= torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model_image_classifier= ImageClassifierCNN(3,10,64).to(device)
loss_fn= nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_image_classifier.parameters(), lr=0.001)
EPOCHS=20

ok now lets bring our training and testing functions, we will copy them as colab is removing their files and importing them is failing

In [11]:
from typing import Callable
from tqdm.auto import tqdm
from timeit import default_timer as timer

def train_model(model:nn.Module,
                data:torch.utils.data.DataLoader,
                lossFn: nn.Module, 
                optimizer:torch.optim.Optimizer,
                epochs:int,
                accr_fun: Callable[[torch.Tensor, torch.Tensor], float],
                device:torch.device):
    model.to(device)

    loss_per_epoch= []
    clock_start= timer()
    dataset_accuracy= []
    for epoch in tqdm(range(epochs)):
        total_correct =0
        total_samples= 0
        true_accuracy= 0
        avg_loss= 0
        model.train()
        for X , y in data:
            X= X.to(device)
            y= y.to(device)
            logit= model(X)
            preds= torch.argmax(logit, dim=1)
            
            
            loss= lossFn(logit, y)

            avg_loss += loss.item()
            total_correct += (preds == y).sum().item()
            total_samples += len(y)
            
            optimizer.zero_grad()
            
            loss.backward()
            
            optimizer.step()
            
            
        avg_loss /= len(data)
        loss_per_epoch.append(avg_loss)
        true_accuracy= total_correct / total_samples
        dataset_accuracy.append(true_accuracy)
    
    clock_stop= timer()
    time_taken= clock_stop - clock_start
        
    return {
        'loss_per_epoch': loss_per_epoch,
        'time_taken' : time_taken,
        'dataset_accuracy': dataset_accuracy
    }
            



def accuracy_function(correct:torch.Tensor, prediction: torch.Tensor)-> float:
    rslt= (correct == prediction).sum().item()
    size= len(correct)
    return rslt/size
 
def test_model(model:nn.Module,
               data:torch.utils.data.DataLoader,
               lossFn: nn.Module,
               device:torch.device):
    model.to(device)
    model.eval()
    total_correct= 0
    total_samples= 0
    total_loss=0
    with torch.inference_mode():
        for X , y in data:
            X= X.to(device)
            y= y.to(device)
            logits= model(X)
            pred= torch.argmax(logits, dim=1)
            total_correct += (y == pred).sum().item()
            total_samples += len(y)
            
            loss= lossFn(logits, y)
            total_loss += loss.item()
            
    accuracy= total_correct / total_samples
    loss= total_loss / len(data)
    return {
        'accuracy': accuracy,
        'loss': loss
    }
    
            

In [12]:
result= train_model(model_image_classifier, train_data_batched,loss_fn, optimizer, EPOCHS,accuracy_function,device)
print(result)

  0%|          | 0/20 [00:00<?, ?it/s]

{'loss_per_epoch': [1.557239819060811, 1.1446424977248892, 0.9442921026283518, 0.816678189217587, 0.7121387140067947, 0.6222156959269053, 0.5550061783674732, 0.48704205649664334, 0.421965798282105, 0.3694604803019625, 0.3251027540801584, 0.2901939593465127, 0.24638946776461723, 0.21921199079498152, 0.19561422292305075, 0.17181865965752194, 0.1471413661561468, 0.14252033700590092, 0.12175814736791699, 0.11059564474703329], 'time_taken': 333.37006972300003, 'dataset_accuracy': [0.42384, 0.59296, 0.6702, 0.71876, 0.75686, 0.78872, 0.81482, 0.8375, 0.85782, 0.87492, 0.89022, 0.9012, 0.91526, 0.9259, 0.9331, 0.94172, 0.9497, 0.95214, 0.958, 0.96312]}


In [13]:
test_results = test_model(model_image_classifier, test_data_batched, loss_fn, device)
print(test_results)

{'accuracy': 0.8548, 'loss': 0.7073405951641167}
